# Data Assimilation: Particle Filters -- Create "observed" data
Femke Vossepoel F.C.Vossepoel@TUDelft.nl; 
Samantha Kim S.S.R.Kim@TUDelft.nl;
Dieter Werthmüller D.Werthmuller@TUDelft.nl

Note: _This script is only to generate "synthetic" data, our **synthetic twin**. For mor info about the forward model **Mogi** please consult the main notebook._

To test the performance of a method, we often conduct a so-called 'synthetic twin' experiment (a.k.a. inverse crime). This means that we generate a (simplified) synthetic model of the Earth, and using an (often also simplified) forward model to generate "artificial" observations. These artificial observations of our synthetic twin is then treated afterwards as if it would be actually observed data. Knowing our artificial world and its behaviour allows us to test a process. After successfully testing it we can apply the process to actually observed data.

Here, we therefore simply generate some data and save it to a 'acquisition-file', together with observation information, in our case $x$ and $y$ coordinates.

## Our model

- Mogi model
- Sources in a horizontally regular grid, 10 km distance between each.
- Some "pseudo-random" depths: -3 km +/- a few dozens meters. *The students will assume -3 km*.
- Some "pseudo-random" strengths between -1 and -5. *The students will start with -2*.
- Observations at the surface at 0 km, same grid as sources.
- Adding Gaussian noise of 1 % to the observations.

*The Gaussian noise plus the wrong depths ensures that one cannot find the true source strengths by manual try-and-error.*

In [1]:
import numpy as np
from mogi_model import mogi
rng = np.random.default_rng()  # Initiate a Random Number Generator

### Sources

In [2]:
# x- and y-coordinates
x = np.array([730000.0, 740000.0, 750000.0])
y = np.array([5920000.0, 5910000.0, 5900000.0])
xy = np.meshgrid(x, y)  # 'meshing'
xx = xy[0].ravel('F')
yy = xy[1].ravel('F')

# Pseudo-random source depths
zz = np.array([-3005.1, -3034.9, -2928.5, -2991.3, -2990.8, -2993.3, -3004.1, -2999.4, -2970.5])

# Assemble coordinates
sources = np.vstack([xx, yy, zz]).transpose()

# Pseudo-random source strengths
strengths = np.array([-2.31, -2.13, -2.56, -1.90, -4.78, -1.26, -1.14, -3.27, -1.43])

print("\n         = Source positions and strengths  =\n\n"
      f"  # ::        x            y          z    strength\n{54*'-'}")
for i in range(sources.shape[0]):
    print(f"{i+1:3d} ::   {sources[i, 0]}    {sources[i, 1]}    {sources[i, 2]}    {strengths[i]}")


         = Source positions and strengths  =

  # ::        x            y          z    strength
------------------------------------------------------
  1 ::   730000.0    5920000.0    -3005.1    -2.31
  2 ::   730000.0    5910000.0    -3034.9    -2.13
  3 ::   730000.0    5900000.0    -2928.5    -2.56
  4 ::   740000.0    5920000.0    -2991.3    -1.9
  5 ::   740000.0    5910000.0    -2990.8    -4.78
  6 ::   740000.0    5900000.0    -2993.3    -1.26
  7 ::   750000.0    5920000.0    -3004.1    -1.14
  8 ::   750000.0    5910000.0    -2999.4    -3.27
  9 ::   750000.0    5900000.0    -2970.5    -1.43


### Observations

Here, we take the same x/y-positions as for the sources, but at z=0. However, we could also choose entirely different locations for the observations (and also much more or much fewer than sources).

In [3]:
obs = np.zeros((sources.shape[0], 4))
obs[:, :2] = sources[:, :2].copy()

# Compute the forward model at the observation points and store it.
obs[:, 3] = mogi(obs[:, 0], obs[:, 1], obs[:, 2], sources, strengths)[2]

# Add Gaussian noise with stddev = 1 %
obs[:, 3] = rng.normal(obs[:, 3], scale=abs(obs[:, 3])*0.01)

# "idealized": Assuming (a) correct strength; (b) depth= -3 km; (c) no noise
idealized = mogi(xx, yy, zz*0, np.vstack([xx, yy, zz*0-3000]).transpose(), strengths)[2]
nrmsd = 200*abs((idealized - obs[:, 3])/(abs(idealized)+abs(obs[:, 3])))

print("\n      = Observation locations and deformations  =\n\n"
      f"  # ::        x            y      z        subsidence      error (%)\n{70*'-'}")
for i in range(obs.shape[0]):
    
    print(f"{i+1:3d} ::   {obs[i, 0]}    {obs[i, 1]}    {obs[i, 2]}    {obs[i, 3]:.6e}       {nrmsd[i]:.2f}")


      = Observation locations and deformations  =

  # ::        x            y      z        subsidence      error (%)
----------------------------------------------------------------------
  1 ::   730000.0    5920000.0    0.0    -6.522483e-02       0.56
  2 ::   730000.0    5910000.0    0.0    -6.202134e-02       2.77
  3 ::   730000.0    5900000.0    0.0    -7.464752e-02       3.81
  4 ::   740000.0    5920000.0    0.0    -5.748421e-02       0.47
  5 ::   740000.0    5910000.0    0.0    -1.319043e-01       1.53
  6 ::   740000.0    5900000.0    0.0    -4.065283e-02       0.13
  7 ::   750000.0    5920000.0    0.0    -3.544879e-02       0.59
  8 ::   750000.0    5910000.0    0.0    -9.323383e-02       0.68
  9 ::   750000.0    5900000.0    0.0    -4.368474e-02       2.61


#### Note
The *idealized* data set assumes that the Mogi sources are taken to be at -3 km, as the students do, and forward modelled without added noise. => **The error is therefore a measure for both the random Gaussian noise added to the observations and the error coming from the wrong depth estimate**.

### Save file

We save the observations to a file called `subsidence_data.txt` in the current directory. This file is loaded as observations in the main Data Assimilation notebook.

In [4]:
np.savetxt(
    'subsidence_data.txt',
    obs,
    fmt='%12.1f %12.1f %8.1f %14.4e',
    delimiter='    ',
    header='   x [m]        y [m]    z [m]   subsidence [m]',
)